##### Copyright 2023 Google LLC

In [ ]:
# !pip install -U -q "google-genai"

# helper

In [19]:
# get current_date_time
from datetime import datetime

def get_doc_title():
  current_date_time = datetime.now()
  # print(current_date_time)

  # concat date_time to string
  current_date_time_str = current_date_time.strftime("%Y%m%d_%H%M%S")
  # print(current_date_time_str)
  return "textbook_" + current_date_time_str

# Init GenAI

In [4]:
from google import genai
from google.genai import types
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

# Explicitly configure the API key using genai.configure
# genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))



client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY"))




# Contents generation


In [5]:
def generate_ai_text(prompt, contents):

  contents = prompt + "\n\n" + contents

  response = client.models.generate_content(
      model='gemini-1.5-flash', contents=contents
  )
  return response.text

# Fox news extraction function

In [6]:
import requests
from bs4 import BeautifulSoup

def extract_fox_news(url):

  print(url)

  return_text = ""

    # 웹 페이지 요청
  response = requests.get(url, verify=True)

  # 페이지의 HTML 콘텐츠 파싱
  soup = BeautifulSoup(response.content, 'html.parser')

  # Get title
  title = soup.find_all('title')
  # print(f"title: {title[0].get_text()}")
  # print(f"{title[0].get_text()}")
  return_text = title[0].get_text() + "\n"

  #Get sub_headline
  sub_headline = soup.find_all("h2", class_="sub-headline")
  # print(f"sub headline : {sub_headline[0].get_text()}")
  # print(f"{sub_headline[0].get_text()}")
  return_text += sub_headline[0].get_text() + "\n"


  # 예시: 헤드라인 뉴스 크롤링
  headlines = soup.find_all('div', class_='article-body')

  for headline in headlines:
      # print_sentences_with_period(headline.get_text())
      # print (headline.get_text())
      return_text += headline.get_text() + "\n"

  return return_text

# Beitbart news extraction function

In [7]:
import requests
from bs4 import BeautifulSoup

def extract_breitbart_news(url):

  print(url)

  return_text = ""

  # 웹 페이지 요청
  response = requests.get(url, verify=True)

  # 페이지의 HTML 콘텐츠 파싱
  soup = BeautifulSoup(response.content, 'html.parser')

  # Get title
  title = soup.find_all('title')
  # print(f"{title[0].get_text()}")

  return_text = title[0].get_text() + "\n"

  #Get sub_headline
  # sub_headline = soup.find_all("p", class_="subheading")
  # print(f"sub headline : {sub_headline[0].get_text()}")
  # for text in sub_headline:
    # print(text.get_text())

  # print("\n")
  # print(f"{sub_headline[0].get_text()}")


  # 예시: 헤드라인 뉴스 크롤링
  headlines = soup.find_all('p')

  for headline in headlines:
      # print_sentences_with_period(headline.get_text())
      # print (headline.get_text())
      return_text += headline.get_text() + "\n"

  return return_text

# Main

In [24]:
FOXNEWS = "foxnews"
BREIBART = "breitbart"

# start
urls = [

    # fox
   "https://www.foxnews.com/world/trumps-un-ambassador-pick-elise-stefanik-could-save-taxpayers-millions-taps-musk-ramaswamy-doge",

  # breibart
    "https://www.breitbart.com/tech/2025/01/14/gamers-claim-elon-musk-paid-for-high-ranked-video-game-account-after-absolutely-clueless-live-performance/"

]

# output dictionary
file_output = ""

for url in urls:

  #file output - copy url
  file_output += url + "\n\n"

  if FOXNEWS in url:
    text = extract_fox_news(url)
  elif BREIBART in url:
    text = extract_breitbart_news(url)
  else:
    print("Error : New Link is not supported yet")

  query = """
     Devide the text into sentences.
     For each sentences, do next action items print the output with below format:
     - English sentence
     - List up IELT’s B2/C1/C2 vocabulrary with korean translation, without separate the words with newline.
     - sentence Korean translation
    """

  query = """
  다음 내용에서 사용된 영어 중급 레벨 이상의 "phrasal verbs"을 찾아줘. 전치사 없이 동사만 사용된 경우는 제외해줘.

  출력 형식은 다음과 같이 해줘.
  - phrasal verb (한글뜻) / 문장속의 예문
  """

  output = generate_ai_text(query,text)
  print(output)

  #file output - copy output
  file_output += output + "\n"

  query = """
다음 영어 텍스트를 분석하여 문장 단위로 나누고, 각 문장에 대해 다음 정보를 각 변수에 담아줘.

1. 영어 문장 (Original English Sentence) -> 변수 : original_english_sentence
2. IELT의 B2/C1/C2 수준의 단어/표현과 한글 뜻 : 변수 : key_words_and_korean_definitions
3. 번역 (Sentence translation in Korean): 변수 : sentence_translation_in_korean

출력 형식은 다음과 같이 해주고, 변수 이름은 출력하지 말고 변수값만 출력.

original_english_sentence

- key_words_and_korean_definitions
-  sentence_translation_in_korean
"""

  output = generate_ai_text(query,text)
  print(output)

  #file output - copy output
  file_output += output + "\n"

file_output += "[End of Document]"

https://www.foxnews.com/world/trumps-un-ambassador-pick-elise-stefanik-could-save-taxpayers-millions-taps-musk-ramaswamy-doge
- **pick** (선택하다) / Trump's UN ambassador pick Elise Stefanik could save taxpayers millions
- **reign in** (억제하다) / an immediate concern for an incoming U.N. ambassador should be reigning in U.S. expenditures
- **step back from** (물러서다) / stepping back from the organization would be counterintuitive
- **swoop in** (급습하다, 갑자기 나타나다) / China is "more than willing to swoop in and fill whatever leadership vacuum we don’t fill
- **look into** (조사하다) / DOGE-U.N. will also look into "where and how the U.N. has been evolving
- **stick with** (고수하다) / DOGE-U.N. is "trying to stick with attacking inefficiencies"
- **turn around** (돌아서다, 방향을 바꾸다) / But as soon as we lose, we turn around and pay for all the lawfare
- **push for** (추진하다) / she stands ready to push for needed reform
- **work through** (해결하다, 극복하다) / working through the confirmation process
- **cover** (지불하다) /

# save to file - google doc

In [9]:
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

from googleapiclient.http import MediaFileUpload

from google.oauth2 import service_account

In [25]:
from googleapiclient.discovery import build

def create_doc(title, content, folder_id=None):
    docs_service = build('docs', 'v1')

    document = docs_service.documents().create(body={'title': title}).execute()
    doc_id = document.get('documentId')

    requests = []
    requests.append({
        'insertText': {
            'location': {
                'index': 1,
            },
            'text': content
        }
    })

    docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()

    # 폴더 지정
    if folder_id:
      file_service = build('drive', 'v3')
      file_service.files().update(fileId=doc_id, addParents=folder_id).execute()

    print(f'Google Docs 파일이 생성되었습니다. ID: {doc_id}')
    return doc_id


text_content = file_output
doc_title = get_doc_title()
folder_id = '1xEcPyhD5K727rmK7ePL0vJneh118YVqK'  # Google Drive 폴더 ID를 넣어주세요

document_id = create_doc(doc_title, text_content, folder_id)

Google Docs 파일이 생성되었습니다. ID: 1gUi7H33i_-flnaPERblHZDzzrfKv7k_dB-GCMi_Tf6Y


*# End of Project*